In [83]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import datefinder
import base64
import email

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = None
def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'gmail_credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            print("These are the creds", creds)
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    results = service.users().messages()
    user_id = "birdyanddoggy@gmail.com"
    query = "free food"
    query_results = results.list(userId= user_id, q=query).execute()
#     msg = query_results["messages"][0]
#     return results.list(userId= user_id, q=query).execute()
#     msg_id =  msg["id"]
#     print(msg_id)
#     message = service.users().messages().get(userId=user_id, id=msg_id,
#                                              format='raw').execute()
    
#     snippet = message["snippet"]
#     msg_str = base64.urlsafe_b64decode(message['raw'].encode('ASCII'))
#     print(msg_str, "\n\n\n\n")
#     mime_msg = email.message_from_bytes(msg_str)

    return query_results


In [146]:
response = main()

In [147]:
def parse_email(message):
    content = ""
    msg_str = base64.urlsafe_b64decode(message['raw'].encode('ASCII'))
    mime_msg = email.message_from_bytes(msg_str)
    for part in mime_msg.walk():
        if(part.get_content_type() == 'text/plain'):
            content = content + " " + part.get_payload()
    return content

def get_email(email_id):
    service = build('gmail', 'v1', credentials=creds)
    message = service.users().messages().get(userId=user_id, id=email_id, format='raw').execute()
    return message

In [161]:
for message in response["messages"]:
    email_obj = get_email(message["id"])
    message["body"] = parse_email(email_obj)

    print(message["id"])


16c8bfc725935780
16bfbb368d4e8c2a
16b0645f5901e4b9
154bb9df10573750
151fdb7d16287c3d
14174c61d84d2e04
140f530732c78b0e
140ef56afc51955e
140e48eb726a96a9
13fe90b6958edefa
13eb59ed8d7e0569
13e873fd12d4d0d8
13e873730d88227a


In [149]:
message["body"]

" We'd like to invite you to an event on Saturday, August 17th 2019 at 6pm.\r\nThere will be free food.\r\n"

In [129]:
response

{'messages': [{'id': '16c8bfc725935780',
   'threadId': '16c8bfc725935780',
   'body': " We'd like to invite you to an event on Saturday, August 17th 2019 at 6pm.\r\nThere will be free food.\r\n"},
  {'id': '16bfbb368d4e8c2a',
   'threadId': '16bfbb368d4e8c2a',
   'body': ' Lucky I saved them on the computer.:)Im trying to be more techy\r\n\r\n\r\nWhat have you done for humankind today?\r\n\r\nSandra Slavin=C2=A0=C2=A0'},
  {'id': '16b0645f5901e4b9', 'threadId': '16b0645f5901e4b9', 'body': ' \r\n'},
  {'id': '154bb9df10573750', 'threadId': '154bb9df10573750', 'body': ' \r\n'},
  {'id': '151fdb7d16287c3d',
   'threadId': '151fdb7d16287c3d',
   'body': ' ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n\r\nFrom All is Well to you!\r\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\r\n\r\nAIW Logo [http://r20.rs6.net/tn.jsp?f=001P9pWpeglMVDuBzDY98cbrP7DEvvhS9yj3i5dl4GOSwP0G6l8RCJxXhBVHCYsfhufZu4Vf0onDAZ3neykmMQaCBf3I4R_4t

In [96]:
get_email("16c8bfc725935780")

{'id': '16c8bfc725935780',
 'threadId': '16c8bfc725935780',
 'labelIds': ['IMPORTANT', 'CATEGORY_PERSONAL', 'INBOX'],
 'snippet': 'We&#39;d like to invite you to an event on Saturday, August 17th 2019 at 6pm. There will be free food.',
 'historyId': '3579559',
 'internalDate': '1565716661000',
 'sizeEstimate': 4948,
 'raw': 'RGVsaXZlcmVkLVRvOiBiaXJkeWFuZGRvZ2d5QGdtYWlsLmNvbQ0KUmVjZWl2ZWQ6IGJ5IDIwMDI6YTI1OmFjODM6MDowOjA6MDowIHdpdGggU01UUCBpZCB4M2NzcDEyMzAwNTF5Ymk7DQogICAgICAgIFR1ZSwgMTMgQXVnIDIwMTkgMTA6MTc6NTMgLTA3MDAgKFBEVCkNClgtUmVjZWl2ZWQ6IGJ5IDIwMDI6YWIwOjYwZDk6OiB3aXRoIFNNVFAgaWQgZzI1bXIxMjkwNTEwNXVhbS42OS4xNTY1NzE2NjczMTExOw0KICAgICAgICBUdWUsIDEzIEF1ZyAyMDE5IDEwOjE3OjUzIC0wNzAwIChQRFQpDQpBUkMtU2VhbDogaT0xOyBhPXJzYS1zaGEyNTY7IHQ9MTU2NTcxNjY3MzsgY3Y9bm9uZTsNCiAgICAgICAgZD1nb29nbGUuY29tOyBzPWFyYy0yMDE2MDgxNjsNCiAgICAgICAgYj1uczBtYndUUWFBZ2lJNm4vWGd1TG9KT1hzWS9kOCtGNmZ3cVZnNTUzZHh2cFp0RW43OURnc0I1MGcreTNrZlFhQjcNCiAgICAgICAgIEhTbHlEOFJCeVVteHRjQ3VQdG1tQmdFQ1VIUDBoQm0zQTgrdnpUcXdvMmNrZ

In [90]:
main()
with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
creds

In [64]:
# query_results = main()
# print(query_results)
# print("message snip ", main())
# matches = datefinder.find_dates(main())

# for match in matches:
#     print("Hello Worlds")
#     start = match
#     print(match)

for part in main().walk():
    # each part is a either non-multipart, or another multipart message
    # that contains further parts... Message is organized like a tree
    if part.get_content_type() == 'text/plain':
        print(part.get_payload()) # prints the raw text



We'd like to invite you to an event on Saturday, August 17th 2019 at 6pm.
There will be free food.



In [124]:
import datetime
user_id = "birdyanddoggy@gmail.com"
query = "free food"


def get_emails(email):
    week_ago = datetime.date.today() - datetime.timedelta(days=7)
    service = build('gmail', 'v1', credentials = self.creds)
    user_id = email
    query = f'free food, after:{week_ago}'
    query_results = service.users().messages().list(userId = user_id, q = query).execute()
    print(query_results)
    return query_results

In [92]:
get_emails("birdyanddoggy@gmail.com")

NameError: name 'self' is not defined

In [31]:
# query_results["messages"][0]
# msg = query_results["messages"][0]
# msg_id = msg["id"]
# msg_id
# print("message snippet ", query_results["snippet"])

In [ ]:
# def create_calendar():
#     """Shows basic usage of the Gmail API.
#     Lists the user's Gmail labels.
#     """
#     creds = None
#     # The file token.pickle stores the user's access and refresh tokens, and is
#     # created automatically when the authorization flow completes for the first
#     # time.
#     if os.path.exists('cal_token.pickle'):
#         with open('cal_token.pickle', 'rb') as token:
#             creds = pickle.load(token)
#     # If there are no (valid) credentials available, let the user log in.
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_secrets_file(
#                 'calendar_credentials.json', 'https://www.googleapis.com/auth/calendar')
#             creds = flow.run_local_server(port=0)
#         # Save the credentials for the next run
#         with open('cal_token.pickle', 'wb') as token:
#             pickle.dump(creds, token)

#     service = build('calendar', 'v3', credentials=creds)
    
#     # calendar api call
#     event = {
#         'start': {
#             'dateTime': '2019-08-17T08:00:00-07:00',
#             'timeZone': 'America/Los_Angeles'
#         },
#         'end': {
#             'dateTime':'2019-08-17T23:00:00-07:00',
#             'timeZone': 'America/Los_Angeles'
#         }
#     }
#     calendar = "birdyanddoggy@gmail.com"
#     return service.events().insert(calendarId=calendar, body=event).execute()
    

In [ ]:
event = create_calendar()
print ('Event created: ', (event.get('htmlLink')))

In [11]:
def quick_add():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    service = build('calendar', 'v3', credentials=creds)
    
    created_event = service.events().quickAdd(
        calendarId='bekkblando@gmail.com',
        text='like to invite you to an event on Saturday, August 17th 2019 at 6pm. There will be free food.').execute()
    return created_event

In [12]:
event = quick_add()
print(event)

{'kind': 'calendar#event', 'etag': '"3131592814334000"', 'id': 'ffr6j3027mmb7nr0cm5g9maip8', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=ZmZyNmozMDI3bW1iN25yMGNtNWc5bWFpcDggYmVra2JsYW5kb0Bt', 'created': '2019-08-14T15:26:47.000Z', 'updated': '2019-08-14T15:26:47.167Z', 'summary': 'like to invite you to an event There will be free food.', 'creator': {'email': 'bekkblando@gmail.com', 'self': True}, 'organizer': {'email': 'bekkblando@gmail.com', 'self': True}, 'start': {'dateTime': '2019-08-17T18:00:00-04:00'}, 'end': {'dateTime': '2019-08-17T19:00:00-04:00'}, 'iCalUID': 'ffr6j3027mmb7nr0cm5g9maip8@google.com', 'sequence': 0, 'reminders': {'useDefault': True}}


In [35]:
class GoogleAPI:

    def __init__(self):
        self.creds = self.auth()
        self.calendar = build('calendar', 'v3', credentials = self.creds)
        self.gmail = build('gmail', 'v1', credentials = self.creds)

    def auth(self):
        creds = None
        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'gmail_credentials.json', 
                    ['https://www.googleapis.com/auth/calendar', 'https://www.googleapis.com/auth/gmail.readonly'],
                    redirect_uri='urn:ietf:wg:oauth:2.0:oob')
                auth_url = flow.authorization_url(prompt='consent')
                print(auth_url)
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                print("This is the credential ", creds)
                pickle.dump(creds, token)
        return creds

        

In [36]:
google_api = GoogleAPI()

('https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=27773405362-pv1pp193r9o5oqj1trnis58calo7s048.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=JapCavwzIDfrVngO93aNKuU0FbxJhi&prompt=consent&access_type=offline&code_challenge=D96V_lk-LuxWf9AeQ0iao2KOuwkULQmuEFEBsWSzQCg&code_challenge_method=S256', 'JapCavwzIDfrVngO93aNKuU0FbxJhi')
This is the credential  None


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [24]:
google_api.calendar.events()